In [1]:
!pip install ultralytics
!pip install supervision==0.14.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.0/869.0 kB 9.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 kB 2.4 MB/s eta 0:00:00


In [2]:
import ultralytics
print("ultralytics.__version__:", ultralytics.__version__)

import supervision
print("supervision.__version__:", supervision.__version__)

import torch
print("torch.__version__:", torch.__version__)

print("CUDA Device:", torch.cuda.get_device_name(0))

ultralytics.__version__: 8.2.77
supervision.__version__: 0.14.0
torch.__version__: 2.1.2
CUDA Device: Tesla T4


# Tracking

In [3]:
from ultralytics import YOLO

video_path = '/kaggle/input/toll-gate/toll_gate.mp4'

# Configure the tracking parameters and run the tracker
model = YOLO('yolov8x.pt')

results = model.track(source=video_path,  classes=[2, 5], conf=0.3, iou=0.5, save=True, tracker="bytetrack.yaml")

100%|██████████| 131M/131M [00:00<00:00, 137MB/s]  


requirements: Ultralytics requirement ['lapx>=0.5.2'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 36.7 MB/s eta 0:00:00a 0:00:01

requirements: AutoUpdate success ✅ 11.7s, installed 1 package: ['lapx>=0.5.2']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/358) /kaggle/input/toll-gate/toll_gate.mp4: 384x640 1 car, 1 bus, 107.0ms
video 1/1 (frame 2/358) /kagg

# Tracking and counting  - Multiple Gate

In [4]:
import cv2
cap = cv2.VideoCapture(video_path)
cap.get(cv2.CAP_PROP_FRAME_WIDTH), cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

(596.0, 336.0)

In [11]:
from collections import defaultdict
import supervision as sv
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8x.pt')

# Set up video capture
cap = cv2.VideoCapture(video_path)

# Define the line coordinates up to 8 gates
START_1, END_1 = sv.Point(10, 135), sv.Point(87, 151)
START_2, END_2 = sv.Point(87, 151), sv.Point(152, 166)
START_3, END_3 = sv.Point(152, 166), sv.Point(219, 180)
START_4, END_4 = sv.Point(219, 180), sv.Point(287, 195)
START_5, END_5 = sv.Point(287, 195), sv.Point(356, 210)
START_6, END_6 = sv.Point(356, 210), sv.Point(428, 226)
START_7, END_7 = sv.Point(390, 245), sv.Point(486, 267)
START_8, END_8 = sv.Point(486, 267), sv.Point(562, 284)

            
# Define the gates
gates = [
    (START_1, END_1),
    (START_2, END_2),
    (START_3, END_3),
    (START_4, END_4),
    (START_5, END_5),
    (START_6, END_6),
    (START_7, END_7),
    (START_8, END_8)
]

# Store the track history
track_history = defaultdict(lambda: [])

# Create a dictionary to keep track of objects that have crossed the line
crossed_objects = [defaultdict(bool) for _ in range(8)]

# Counters for cars and buses
car_count = 0
bus_count = 0

# Open a video sink for the output video
video_info = sv.VideoInfo.from_video_path(video_path)
with sv.VideoSink("output_multi_gate_vehicle_detection.mp4", video_info) as sink:

    while cap.isOpened():
        success, frame = cap.read()

        if success:
            # Run YOLOv8 tracking on the frame, persisting tracks between frames
            results = model.track(frame,  classes=[2, 5], conf=0.3, iou=0.5, persist=True, tracker="bytetrack.yaml")

            # Get the boxes and track IDs
            boxes = results[0].boxes.xywh.cpu()
            track_ids = results[0].boxes.id.int().cpu().tolist()
            class_ids = results[0].boxes.cls.int().cpu().tolist()

            # Visualize the results on the frame
            annotated_frame = results[0].plot()
            detections = sv.Detections.from_ultralytics(results[0])

            # Plot the tracks and count objects crossing the line
            for box, track_id, class_id in zip(boxes, track_ids, class_ids):
                x, y, w, h = box
                track = track_history[track_id]
                track.append((float(x), float(y)))  # x, y center point
                if len(track) > 30:  # retain 30 tracks for 30 frames
                    track.pop(0)

                for i, (start, end) in enumerate(gates):
                    # Check if the object crosses the line
                    if start.x < x < end.x and start.y < y < end.y:
                        # Mark the object as crossed
                        if not crossed_objects[i][track_id]:
                            crossed_objects[i][track_id] = True
                            # add the counters based on the class ID
                            if class_id == 2:  # Car
                                car_count += 1
                            elif class_id == 5:  # Bus
                                bus_count += 1

                        # Annotate the object as it crosses the line
                        cv2.rectangle(annotated_frame, (int(x - w / 2), int(y - h / 2)),
                                        (int(x + w / 2), int(y + h / 2)), (0, 255, 0), 2)
                        
            # Draw the line on the frame
            for start, end in gates:
                cv2.line(annotated_frame, (start.x, start.y), (end.x, end.y), (0, 255, 0), 2)

            # Label the gates
            for i, xpos in enumerate([50, 115, 180, 250, 315, 380, 460, 535]):
                gate_text = f"{i+1}"
                cv2.putText(annotated_frame, gate_text, (xpos, int((60771 + 125*xpos)/569)),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)

            # Write the number of Vehicle get through the gate
            for i, ypos in enumerate(range(20, 100, 10)):
                count_text = f"Objects crossed gate {i+1}: {len(crossed_objects[i])}"
                cv2.putText(annotated_frame, count_text, (10, ypos), cv2.FONT_HERSHEY_SIMPLEX,
                            0.5, (0, 0, 0), 1, cv2.LINE_AA)
            
            # Display the vehicle counts on the frame
            cv2.putText(annotated_frame, f"Cars: {car_count}", (310, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 0, 0), 2, cv2.LINE_AA)
            cv2.putText(annotated_frame, f"Buses: {bus_count}", (310, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (255, 0, 0), 2, cv2.LINE_AA)

            # Write the frame with annotations to the output video
            sink.write_frame(annotated_frame)
        else:
            break

# Release the video capture
cap.release()


0: 384x640 1 car, 1 bus, 61.3ms
Speed: 1.8ms preprocess, 61.3ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 bus, 59.4ms
Speed: 2.2ms preprocess, 59.4ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 bus, 44.6ms
Speed: 1.4ms preprocess, 44.6ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 bus, 33.9ms
Speed: 1.6ms preprocess, 33.9ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 bus, 33.6ms
Speed: 1.5ms preprocess, 33.6ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 bus, 33.3ms
Speed: 2.1ms preprocess, 33.3ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 bus, 34.7ms
Speed: 1.6ms preprocess, 34.7ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 car, 1 bus, 34.0ms
Speed: 2.1ms preprocess, 34.0ms inference, 1.3ms postproc